## Win/Loss Rating Model Prediction

Load the model and make predictions

In [1]:
import requests
import pandas as pd
import numpy as np
import pymc3 as pm
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
from spcl_case import *
plt.style.use('fivethirtyeight')
%matplotlib inline

### Get a list of all CS Games

In [2]:
r = requests.get('https://thunderpick.com/api/matches')
games = pd.DataFrame(r.json()['data'])
games = games[games.gameId == 6].sort_values('championship')

bet_games = []
for i,v in games.iterrows():
    if((v['isTournament'] == False )& (v['canWager'] == True)):
        ratio = v['matchBet']['buckets'][0]['amount']/v['matchBet']['buckets'][1]['amount']
        odds = (ratio**-1+1, ratio+1)
        wr = (odds[1]/np.sum(odds)*100., odds[0]/np.sum(odds)*100.)
        det = requests.get('https://thunderpick.com/api/matches/'+str(v['id'])).json()['data']
        print('Date: %s  |  Event: %s  | (BO%s) %s vs. %s  |  (%.1f:%.1f) | Total Coins: %i' % 
              (v['startTime'][:10], v['championship'], det['bestOfMaps'], v['matchBet']['buckets'][0]['label'], 
               v['matchBet']['buckets'][1]['label'], wr[0], wr[1], v['matchBet']['amount']))
        bet_games.append({'1': v['matchBet']['buckets'][0]['label'], '2': v['matchBet']['buckets'][1]['label'], 'bo': det['bestOfMaps'], 'o1': odds[0], 'o2': odds[1], 'wr': wr[0]})
bet_games = pd.DataFrame(bet_games)

Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) Ghost vs. Immortals  |  (37.0:63.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) EnVyUs vs. fnatic  |  (52.0:48.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) fnatic vs. EnVyUs  |  (45.0:55.0) | Total Coins: 2000
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) FaZe vs. mousesports  |  (63.2:36.8) | Total Coins: 10324
Date: 2017-10-25  |  Event: ECS Season 4 Development League  | (BO1) mousesports vs. FaZe  |  (36.1:63.9) | Total Coins: 2050
Date: 2017-10-24  |  Event: ECS Season 4 Development League  | (BO1) G2 vs. Heroic  |  (68.5:31.5) | Total Coins: 2540
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) Liquid vs. SK  |  (40.0:60.0) | Total Coins: 10000
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) SK vs. Liquid  |  (62.8:37.2) | Total Coins: 2150
Date: 2017-10-25  

## Load Ratings Model

In [54]:
TEAM_SET = 'mdlna'

teams = np.load('saved_model/'+TEAM_SET+'/teams.npy')
maps = np.load('saved_model/'+TEAM_SET+'/maps.npy')
filt = np.load('saved_model/'+TEAM_SET+'/filter_teams.npy')
h_teams = pd.read_csv('hltv_csv/teams.csv').set_index('ID').loc[teams]
h_teams = fix_teams(h_teams)

h_teams_filt = h_teams[h_teams.Name.isin(filt)]

def prep_pymc_model(n_teams, n_maps):
    with pm.Model() as rating_model:
        omega = pm.HalfCauchy('omega', 0.5)
        tau = pm.HalfCauchy('tau', 0.5)
        rating = pm.Normal('rating', 0, omega, shape=n_teams)
        theta_tilde = pm.Normal('rate_t', mu=0, sd=1, shape=(n_maps, n_teams))
        rating_map = pm.Deterministic('rating | map', rating + tau * theta_tilde)
        alpha = pm.Gamma('alpha', 10, 5)
        sigma = pm.HalfCauchy('sigma', 0.5, shape=n_maps)
    return rating_model

rating_model = prep_pymc_model(len(teams), len(maps))
trace = pm.backends.text.load('saved_model/'+TEAM_SET+'/trace', model=rating_model)

## Ban/Pick Predictions

In [55]:
h_bp = pd.read_csv('hltv_csv/picksAndBans.csv').set_index('Match ID')
h_matches = pd.read_csv('hltv_csv/matchResults.csv').set_index('Match ID')
h_matches['Date'] = pd.to_datetime(h_matches['Date'])
h_matches = h_matches[h_matches['Date'] >= dt.datetime(2017,1,1)]
h_bp = h_bp.join(h_matches[['Date']], how='left')
h_bp['Date'] = pd.to_datetime(h_bp['Date'])
h_bp = h_bp[h_bp['Date'] >= dt.datetime(2017,1,1)]

In [56]:
def model_mp(train, t1, t2):
    tab = train[train['Team'].isin([t1, t2])].groupby(['Team', ' Pick Type', 'Map'])['Date'].count().unstack([' Pick Type', 'Team']).fillna(0)
    return (tab/tab.sum(axis=0)).mean(level=0,axis=1)# get average

def model_played(train, t1, t2):
    a = train[train['Team 1 ID'].isin([t1,t2])].groupby(['Team 1 ID', 'Map'])['Date'].count()
    b = train[train['Team 2 ID'].isin([t1,t2])].groupby(['Team 2 ID', 'Map'])['Date'].count()
    c = pd.DataFrame([a,b], index=['a','b']).T.fillna(0)
    c = (c['a']+c['b']).unstack(level=0).fillna(0)
    return (c/c.sum()).mean(axis=1)

def predict_map(func, data, t1, t2):
    res = func(data, t1, t2)
    return res.loc[res.index != 'Default'].sort_values(ascending=False)

# Bet Predictions

In [57]:
money = 4500.
bet_games['1'] = bet_games['1'].str.replace('ex-Denial', 'Denial')
bet_games['2'] = bet_games['2'].str.replace('ex-Denial', 'Denial')
matches = bet_games[bet_games['1'].isin(filt) & bet_games['2'].isin(filt)].drop_duplicates()
def sig(x):
    return 1 / (1 + np.exp(-x))
def abs_norm_interval(start,end,loc,scale):
    return (norm.cdf(end,loc,scale) - norm.cdf(start,loc,scale)) + (norm.cdf(-1*start,loc,scale) - norm.cdf(-1*end,loc,scale))

t_rating = trace['rating']
t_map_rating = trace['rating | map']
t_alpha = trace['alpha']
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    trace_1 = t_rating[:,t1_ind]; trace_2 = t_rating[:,t2_ind]
    mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
    diff = trace_1-trace_2
    p_wl = sig(diff)
    wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
    kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
    kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
    print('%s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
          (v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
           kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))

Iceberg (-0.245) vs ANTI ECO (-1.202) - I:60.00% | P:42.17% - 90.23%  -  K: 4.7% (211) - 0.6% (28)
GX (2.920) vs Mythic (0.923) - I:63.93% | P:66.75% - 96.31%  -  K: 7.5% (336) - -1.5% (-68)
ex-Nitrious (-0.123) vs Rise Nation (2.571) - I:22.68% | P:2.15% - 17.40%  -  K: -1.6% (-72) - 9.1% (407)
FRENCH CANADIANS (1.579) vs Gale Force (1.863) - I:53.33% | P:20.84% - 67.66%  -  K: 0.5% (23) - 4.4% (198)
Denial (1.820) vs FRENCH CANADIANS (1.579) - I:40.00% | P:33.40% - 77.06%  -  K: 3.3% (148) - 1.7% (74)
GX (2.920) vs SoaR (1.345) - I:65.96% | P:55.63% - 94.96%  -  K: 6.5% (292) - -0.6% (-25)
Gale Force (1.863) vs Beacon (0.203) - I:60.00% | P:64.69% - 93.65%  -  K: 7.1% (317) - -1.4% (-63)
LFAO (0.117) vs ANTI ECO (-1.202) - I:60.00% | P:52.40% - 92.69%  -  K: 6.0% (268) - -0.5% (-21)
Mythic (0.923) vs Torqued (1.340) - I:44.19% | P:18.19% - 65.46%  -  K: 0.6% (26) - 4.6% (206)
Adaptation (-0.732) vs Rise Nation (2.571) - I:40.00% | P:1.30% - 9.82%  -  K: -3.6% (-161) - 9.5% (429)
Beac

In [58]:
PRINT_RD_DIFF = False
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    pred_maps = predict_map(model_played, h_matches, t1_id, t2_id)
    pred_maps = pred_maps/pred_maps.sum()
    for m,s in pred_maps.iteritems():
        m_ind = np.where(maps == m)[0][0]
        trace_1 = t_map_rating[:,m_ind,t1_ind]; trace_2 = t_map_rating[:,m_ind,t2_ind]
        mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
        diff = trace_1-trace_2
        p_wl = sig(diff)
        wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
        kappa = 32*sig(t_alpha*diff)-16.
        kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
        kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
        print('    Map: %s (%.2f)  -  %s (%.3f) vs %s (%.3f) - I:%.2f%% | P:%.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
             (m, s*100., v['1'], mr_1, v['2'], mr_2, v['wr'], wr_25*100, wr_75*100, kelly_pct_1*100., 
               kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))
        
        if(PRINT_RD_DIFF):
            p_sc = [abs_norm_interval(x[0],x[1],kappa,trace['sigma'][:,m_ind]) for x in [[1.5,3.5],[3.5,5.5],[5.5,7.5],[7.5,9.5],[9.5,16]]]
            for i,sd in enumerate(['2 - 3 Rounds', '4 - 5 rounds', '6 - 7 rounds', '8 - 9 rounds', '10 rounds or more']):
                sc_25 = np.percentile(p_sc[i], 25); sc_75 = np.percentile(p_sc[i], 75)
                print('      %s : %.2f%% - %.2f%%' % (sd, sc_25*100, sc_75*100))

    Map: Mirage (37.80)  -  Iceberg (-1.188) vs ANTI ECO (-2.318) - I:60.00% | P:49.70% - 90.15%  -  K: 5.4% (242) - 0.0% (1)
    Map: Train (18.90)  -  Iceberg (0.816) vs ANTI ECO (2.658) - I:60.00% | P:3.49% - 42.49%  -  K: -4.2% (-188) - 8.4% (378)
    Map: Inferno (15.07)  -  Iceberg (1.745) vs ANTI ECO (-1.842) - I:60.00% | P:88.32% - 99.36%  -  K: 9.3% (420) - -3.4% (-154)
    Map: Overpass (10.53)  -  Iceberg (-0.289) vs ANTI ECO (-0.429) - I:60.00% | P:10.52% - 91.33%  -  K: 0.9% (40) - 4.0% (178)
    Map: Cache (9.81)  -  Iceberg (-2.636) vs ANTI ECO (-1.804) - I:60.00% | P:7.53% - 69.95%  -  K: -2.2% (-97) - 6.6% (298)
    Map: Nuke (5.26)  -  Iceberg (-0.252) vs ANTI ECO (-2.384) - I:60.00% | P:37.05% - 99.13%  -  K: 7.2% (321) - -1.5% (-67)
    Map: Cobblestone (2.63)  -  Iceberg (-0.264) vs ANTI ECO (-3.485) - I:60.00% | P:66.66% - 99.67%  -  K: 9.0% (406) - -3.2% (-142)
    Map: Mirage (33.75)  -  GX (3.783) vs Mythic (1.780) - I:63.93% | P:72.25% - 95.18%  -  K: 7.6% (34

    Map: Overpass (9.87)  -  Mythic (-1.069) vs Torqued (0.025) - I:44.19% | P:7.40% - 58.17%  -  K: -1.5% (-68) - 6.9% (308)
    Map: Nuke (3.62)  -  Mythic (1.764) vs Torqued (1.353) - I:44.19% | P:13.45% - 93.49%  -  K: 2.7% (122) - 2.3% (102)
    Map: Dust2 (0.72)  -  Mythic (2.019) vs Torqued (1.316) - I:44.19% | P:13.36% - 96.45%  -  K: 3.5% (159) - 1.4% (62)
    Map: Mirage (23.89)  -  Adaptation (-0.088) vs Rise Nation (2.877) - I:40.00% | P:2.51% - 9.54%  -  K: -3.4% (-152) - 9.3% (418)
    Map: Inferno (22.10)  -  Adaptation (-0.978) vs Rise Nation (4.515) - I:40.00% | P:0.19% - 0.96%  -  K: -3.9% (-177) - 9.9% (447)
    Map: Train (16.57)  -  Adaptation (-0.229) vs Rise Nation (4.127) - I:40.00% | P:0.51% - 3.26%  -  K: -3.8% (-173) - 9.8% (442)
    Map: Cobblestone (16.17)  -  Adaptation (-0.075) vs Rise Nation (2.975) - I:40.00% | P:1.65% - 11.65%  -  K: -3.5% (-155) - 9.4% (421)
    Map: Overpass (9.94)  -  Adaptation (-1.193) vs Rise Nation (1.199) - I:40.00% | P:2.81% -

    Map: Mirage (24.93)  -  Naventic (1.427) vs Muffin Lightning (2.235) - I:45.00% | P:15.59% - 53.03%  -  K: -0.5% (-20) - 5.7% (255)
    Map: Inferno (18.48)  -  Naventic (-3.141) vs Muffin Lightning (0.125) - I:45.00% | P:1.14% - 11.19%  -  K: -4.1% (-183) - 9.5% (429)
    Map: Cache (14.88)  -  Naventic (4.695) vs Muffin Lightning (0.083) - I:45.00% | P:96.23% - 99.74%  -  K: 9.8% (441) - -5.3% (-238)
    Map: Cobblestone (12.61)  -  Naventic (0.587) vs Muffin Lightning (1.476) - I:45.00% | P:9.93% - 63.42%  -  K: -0.6% (-29) - 5.9% (264)
    Map: Overpass (10.04)  -  Naventic (5.356) vs Muffin Lightning (0.564) - I:45.00% | P:95.01% - 99.86%  -  K: 9.8% (442) - -5.3% (-239)
    Map: Dust2 (8.06)  -  Naventic (0.835) vs Muffin Lightning (2.486) - I:45.00% | P:2.21% - 62.12%  -  K: -2.7% (-121) - 8.1% (363)
    Map: Train (6.16)  -  Naventic (-3.347) vs Muffin Lightning (1.611) - I:45.00% | P:0.11% - 4.53%  -  K: -4.4% (-199) - 9.9% (446)
    Map: Nuke (4.84)  -  Naventic (0.895) v

In [34]:
m

'Default'

In [ ]:
plt.ylim(0,1.2)
sns.kdeplot(trace_1, shade=True, alpha=0.65, legend=True, label=v['1'])
sns.kdeplot(trace_2, shade=True, alpha=0.65, legend=True, label=v['2'])

In [ ]:
h_bp.groupby('Match ID').first().count()

In [ ]:
h_bp